In [1]:
import json 

def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)
    
def save_json(filename, obj):
    with open(filename, "w+", encoding="utf-8") as f:
        return json.dump(obj, f, ensure_ascii=False, indent=4) 
    
def load_file(filename):
    return list(open(filename, encoding="utf-8").readlines())


def del_none(d):
    for key, value in list(d.items()):
        if value is None or value == "":
            del d[key]
        elif isinstance(value, dict):
            del_none(value)
    return d

In [ ]:
event_types = load_json("../data/unified.json")["types"]

In [2]:
tut = load_json("../data/tutby.json")
cd = load_json("../data/citydog.json")
relax = load_json("../data/relax.json")
old_events = load_json("../data/events.json")

events = []
events.extend(tut)
events.extend(cd)
events.extend(relax)
events.extend(old_events)

In [4]:
len([event['place'] for event in events if 'address' not in event['place']])

0

In [10]:
places = {}
for event in events:
    place = event['place']
    address = place['address']
    if address not in places:
        places[address] = set()
    places[address].add(place['name'])
for key, value in places.items():
    places[key] = list(places[key])

In [13]:
save_json("../data/places.json", places)

In [ ]:
tg_vk_events = []
for e in load_file("../labeled_text"):
    labeled_event = json.loads(e)
    delabled_event = {"text": labeled_event["text"], "labels": []}
    for start, end, label in labeled_event["labels"]:
        delabled_event["labels"].append((labeled_event["text"][start:end], label))
    tg_vk_events.append(delabled_event)

In [11]:
import re
import urlextract
url_extractor = urlextract.URLExtract()
        
freq = [
    "работая с артистами, которых ранее никому не удавалось привезти в Беларусь -",
    "Минские организаторы, не перестают радовать Белорусских фанатов,",
    "Минские организаторы, не перестают радовать фанатов,",
    "Baby Tape, Boulevard Depo, Loqiemean, Jeembo &",
    "Билеты можно приобрести онлайн на AFISHA.TUT.BY",
    "Билеты можно купить онлайн на AFISHA.TUT.BY",
    "Фото и видео",
    "Depo, Loqiemean, Jeembo &",
    "на AFISHA.TUT.BY Фото и видео",
    "О событии"
]

space_re = re.compile(r"[\s|\u2800]+")
comma_re = re.compile(r"(([^\d])\s*,\s*(\w))")
comma_re_2 = re.compile(r"((\w)\s*,\s*([^\d]))")
dash_re = re.compile(r"(([^\d])\s*\-+\s*([^\d]))")
underscore_re = re.compile(r"\_+")
hashtag_re = re.compile(r"#\w+")


def remove_urls(text):
    urls = list(set(url_extractor.find_urls(text)))
    urls.sort(key=lambda u: len(u), reverse=True)
    for url in urls:
        text = text.replace(url, "")
    return text

def preprocess(text):
    text = text.replace("\n", " ")
    text = remove_urls(text)
    text = hashtag_re.sub("", text)
    text = underscore_re.sub(" ", text)
    for f in freq:
        text = text.replace(f, "")
    for whole, first, second in reversed(comma_re.findall(text)):
        text = text.replace(whole, "%s, %s" % (first, second))
    for whole, first, second in reversed(comma_re_2.findall(text)):
        text = text.replace(whole, "%s, %s" % (first, second))
    text = space_re.sub(" ", text)
    return text.strip()

In [ ]:
for e in events:
    text = e['description']
    e['description'] = preprocess(text)

In [13]:
def detect_lang(text):
    by_letters = ['і', 'ў', 'шч']
    for by_letter in by_letters:
        if by_letter in text:
            return "by"
    return "ru"

def is_suitable_text(text):
    if detect_lang(text) == "by":
        return False
    if len(text) > 2000 or len(text) < 80:
        return False
    if all(not c.isdigit() for c in text):
        return False
    return True

In [14]:
events = [e for e in events if is_suitable_text(e['description'])]

In [34]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
import numpy as np

def remove_duplicated_texts(evs):
    removed_idx = set()
    unique_events = []
    for i, current_text in enumerate(evs):
        if i in removed_idx:
            continue
            
        duplicates = [current_text]
        for j, text in enumerate(evs[i + 1:]):
            if j in removed_idx:
                continue
            res = similar(current_text, text)
            if res > 0.6:
                duplicates.append(text)
                removed_idx.add(j)
                
        unique_event = max(duplicates, key=lambda e: len(e))
        unique_events.append(unique_event)
        
    return unique_events

def remove_duplicated(evs):
    removed_idx = set()
    unique_events = []
    for i, current_event in enumerate(evs):
        if i in removed_idx:
            continue
            
        duplicates = [current_event]
        for j, event in enumerate(evs[i + 1:]):
            if j in removed_idx:
                continue
            res = similar(current_event['description'], event['description'])
            if res > 0.6:
                duplicates.append(event)
                removed_idx.add(j)
                
        unique_event = max(duplicates, key=lambda e: len(e['description']))
        unique_events.append(unique_event)
        
    return unique_events

In [7]:
import random
import datetime

def group_by_dates(events):
    events_by_dates = {}
    for event in events:
        date = event['dates'][0]
        if 'start' in date:
            date = date['start']
        if date:
            if "hour" in date:
                dt = datetime.datetime(year=date['year'], month=date['month'], day=date['day'], hour=date['hour'], minute=date['minute'])
            else:
                dt = datetime.date(year=date['year'], month=date['month'], day=date['day'])
        else:
            dt = datetime.date(year=datetime.MINYEAR, month=1, day=1)

        if dt not in events_by_dates:
            events_by_dates[dt] = []
        events_by_dates[dt].append(event)
    return events_by_dates

def group_by_place(events):
    events_by_place = {}
    for event in events:
        place = event['place']['name']
        if place not in events_by_place:
            events_by_place[place] = []
        events_by_place[place].append(event)
    return events_by_place

In [ ]:
filtered_events = []
for dt, grouped_events in events_by_dates.items():
    initial_len = len(grouped_events)
    if initial_len > 1:
        grouped_events = remove_duplicated(grouped_events)
    filtered_events.extend(grouped_events)

In [ ]:
import random

random.shuffle(filtered_events)
len(filtered_events)

In [ ]:
events_by_types = {}
for event in filtered_events:
    tags = event['tags']
    for tag in tags:
        if tag not in event_types:
            continue
        
        if tag not in events_by_types:
            events_by_types[tag] = []
        
        if len(events_by_types[tag]) > 200:
            continue
        events_by_types[tag].append(event)

            
reduced_events = []
for _, grouped_event in events_by_types.items():
    reduced_events.extend(grouped_event)
random.shuffle(reduced_events)
len(reduced_events)

In [ ]:
for item in reduced_events:
    if "dates" in item:
        del item["dates"]
    if "poster" in item:
        del item["poster"]
    if "url" in item:
        del item["url"]
    del_none(item)

In [ ]:
save_json("../data/reduced_events.json", reduced_events)

In [ ]:
for event in tg_vk_events:
    event["text"] = preprocess(event["text"])
    
tg_vk_events = [e for e in tg_vk_events if is_suitable_text(e["text"])]

In [ ]:
len(tg_vk_events)

In [ ]:
tg_vk_event_pairs = [(e, e["text"][:50]) for e in tg_vk_events]
short = [p[1] for p in tg_vk_event_pairs]
short = set(remove_duplicated_texts(short))
tg_vk_events = [e for e, short_e in tg_vk_event_pairs if short_e in short]

In [ ]:
save_json("../data/reduced_events_social.json", tg_vk_events)

In [ ]:
with open("labeled_text", "w+", encoding="utf-8") as f:
    for event in tg_vk_events:
        text = event["text"]
        labeled = {"text": text, "labels": []}
        for string, label in event["labels"]:
            start = text.find(string)
            if start < 0:
                continue
            end = start + len(string)
            labeled["labels"].append((start, end, label))
        f.write(json.dumps(labeled, ensure_ascii=False) + "\n")

In [ ]:
def count_n_grams(n, texts):
    n_grams = {}
    for text in texts:
        for line in text.split('\n'):
            tokens = line.split(' ')
            for i in range(len(tokens) - n + 1):
                n_gram = " ".join(tokens[i:i + n])
                if n_gram not in n_grams:
                    n_grams[n_gram] = 0
                n_grams[n_gram] += 1
    return n_grams

def get_n_grams(n, text):
    tokens = text.split(' ')
    for i in range(len(tokens) - n + 1):
        n_gram = " ".join(tokens[i:i + n])
        yield n_gram

def get_top_n(n_grams, top):
    return sorted([(g, c) for g, c in n_grams.items()], key=lambda p: p[1], reverse=True)[:top]

def combine_tokens(grams):
    grams = list(grams)
    while len(grams) > 0:
        g = grams.pop()
        toks = g.split(' ')
        found = False
        for i, gr in enumerate(grams):
            gr_toks = gr.split(' ')
            if toks[len(toks) - 3:] == gr_toks[:3]:
                common = toks
                common.extend(gr_toks[3:])
                grams.append(" ".join(common))
                del grams[i]
                found = True
                break
            elif toks[:3] == gr_toks[len(gr_toks) - 3:]:
                common = gr_toks
                common.extend(toks[3:])
                grams.append(" ".join(common))
                del grams[i]
                found = True
                break
        if not found:
            yield g
            
def analyze_texts(texts):
    top_20_tri = get_top_n(count_n_grams(5, texts), 200)
    buckets = {}
    for g, c in top_20_tri:
        if c not in buckets:
            buckets[c] = []
        buckets[c].append(g)

    for count, grams in buckets.items():
        if count < 20:
            continue
        combined = list(combine_tokens(grams))
        print(count)
        for gram in sorted(combined, key=lambda p: len(p), reverse=True):
            if len(gram.split(' ')) < 6:
                continue
            print(gram)

In [ ]:
import difflib

def find_place(text, place):
    place_grams = len(place.split(' '))
    matches = []
    for n in range(1, place_grams * 2):
        grams = list(get_n_grams(n, text))
        for gram in grams:
            rate = similar(place, gram)
            if rate > 0.8:
                matches.append((gram, rate))
    if not matches:
        return None
    match = max(matches, key=lambda p: p[1])[0]
    return match

In [ ]:
for event in reduced_events:
    text = event["description"]
    title = event["title"]
    match = find_place(text, title)
    if match:
        print(title)
        print(match)
        print()

In [ ]:
reduced_events[163]

In [ ]:
embeddings = {}
with open("") as f:
    for line in f:
        word, *vector = line.split(',')
        vector = [float(v) for v in vector]
        embeddings[word] = np.array(vector)
        
def get_vector(word):
    if word in embeddings:
        return embeddings[word]
    new_vector = np.random.rand(2048)
    embeddings[word] = new_vector
    return new_vector

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
from nltk.tokenize import word_tokenize


from pymystem3 import Mystem
from string import punctuation

mystem = Mystem()
def preprocess_text(text):
    tokens = word_tokenize(text.lower()) #mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    #text = " ".join(tokens)
    
    return tokens

In [18]:
import sys
import re
import os
import numpy as np
import tensorflow as tf
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
from sklearn import preprocessing


def tokenize(string):
    """
    :param string: well, text string
    :return: list of tokens
    """
    token_pattern = re.compile('(?u)\w+')
    tokens = [t.lower() for t in token_pattern.findall(string)]
    return tokens


def get_elmo_vectors(sess, texts, batcher, sentence_character_ids, elmo_sentence_input):
    """
    :param sess: TensorFlow session
    :param texts: list of sentences (lists of words)
    :param batcher: ELMo batcher object
    :param sentence_character_ids: ELMo character id placeholders
    :param elmo_sentence_input: ELMo op object
    :return: embedding matrix for all sentences (max word count by vector size)
    """

    # Create batches of data.
    sentence_ids = batcher.batch_sentences(texts)
    print('Sentences in this batch:', len(texts), file=sys.stderr)

    # Compute ELMo representations.
    elmo_sentence_input_ = sess.run(elmo_sentence_input['weighted_op'],
                                    feed_dict={sentence_character_ids: sentence_ids})

    return elmo_sentence_input_


def get_elmo_vector_average(sess, texts, batcher, sentence_character_ids, elmo_sentence_input):
    vectors = []

    # Create batches of data.
    sentence_ids = batcher.batch_sentences(texts)
    print('Sentences in this chunk:', len(texts), file=sys.stderr)
    # Compute ELMo representations.
    elmo_sentence_input_ = sess.run(elmo_sentence_input['weighted_op'],
                                    feed_dict={sentence_character_ids: sentence_ids})
    print('ELMo sentence input shape:', elmo_sentence_input_.shape, file=sys.stderr)
    for sentence in range(len(texts)):
        sent_vec = np.zeros((elmo_sentence_input_.shape[1], elmo_sentence_input_.shape[2]))
        for word_vec in enumerate(elmo_sentence_input_[sentence, :, :]):
            sent_vec[word_vec[0], :] = word_vec[1]
        semantic_fingerprint = np.sum(sent_vec, axis=0)
        semantic_fingerprint = np.divide(semantic_fingerprint, sent_vec.shape[0])
        query_vec = preprocessing.normalize(semantic_fingerprint.reshape(1, -1), norm='l2')
        vectors.append(query_vec.reshape(-1))
    return vectors


def load_elmo_embeddings(directory, top=False):
    """
    :param directory: directory with an ELMo model ('model.hdf5', 'options.json' and 'vocab.txt.gz')
    :param top: use ony top ELMo layer
    :return: ELMo batcher, character id placeholders, op object
    """
    if os.path.isfile(os.path.join(directory, 'vocab.txt.gz')):
        vocab_file = os.path.join(directory, 'vocab.txt.gz')
    elif os.path.isfile(os.path.join(directory, 'vocab.txt')):
        vocab_file = os.path.join(directory, 'vocab.txt')
    else:
        raise SystemExit('Error: no vocabulary file found in the directory.')
    options_file = os.path.join(directory, 'options.json')
    weight_file = os.path.join(directory, 'model.hdf5')

    # Create a Batcher to map text to character ids.
    batcher = Batcher(vocab_file, 50)

    # Input placeholders to the biLM.
    sentence_character_ids = tf.compat.v1.placeholder('int32', shape=(None, None, 50))

    # Build the biLM graph.
    bilm = BidirectionalLanguageModel(options_file, weight_file, max_batch_size=128)

    # Get ops to compute the LM embeddings.
    sentence_embeddings_op = bilm(sentence_character_ids)

    # Get an op to compute ELMo (weighted average of the internal biLM layers)
    elmo_sentence_input = weight_layers('input', sentence_embeddings_op, use_top_only=top)
    return batcher, sentence_character_ids, elmo_sentence_input


def divide_chunks(data, n):
    for i in range(0, len(data), n):
        yield data[i:i + n]

In [15]:
sentences = [
    'Состоится концерт группы The Feedback',
    'Группа The Feedback отыграет концерт',
    'Приходите на концерт группы The Feedback',
    'Концерт группы The Beatles',
    'Концерт группы The Feedback'
]
sentences = [tokenize(s) for s in sentences]

In [19]:
batcher, sentence_character_ids, elmo_sentence_input = load_elmo_embeddings("C:/Users/ASUS/Downloads/199")

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
def get_vectors(sentences):
    with tf.compat.v1.Session() as sess:
        # It is necessary to initialize variables once before running inference.
        sess.run(tf.compat.v1.global_variables_initializer())
        elmo_vectors = get_elmo_vectors(sess, sentences, batcher, sentence_character_ids, elmo_sentence_input)
        return elmo_vectors

In [35]:
def print_heat_matrix(vectors):
    size = len(vectors)
    heat_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i == j:
                heat_matrix[i][j] = 0
            else:
                heat_matrix[i][j] = compute_distance(vectors[i], vectors[j])
    print(heat_matrix)
    
def print_heat_matrix_text(sentences):
    size = len(sentences)
    heat_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i == j:
                heat_matrix[i][j] = 0
            else:
                heat_matrix[i][j] = similar(sentences[i], sentences[j])
    print(heat_matrix)

In [26]:
import scipy

def compute_distance(vector1, vector2):
    vector_1 = np.mean(vector1, axis=0)
    vector_2 = np.mean(vector2, axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return cosine

def count_heat(vector_1, vector_2):
    heat_matrix = np.zeros((len(vector_1), len(vector_2)))
    for i in range(len(vector_1)):
        v1 = vector_1[i]
        for j in range(len(vector_2)):
            v2 = vector_2[j]
            heat_matrix[i][j] = round(scipy.spatial.distance.cosine(v1, v2), 3)
    return heat_matrix

In [21]:
count_heat(elmo_vectors[0], elmo_vectors[1])

array([[0.664, 0.786, 0.721, 0.516, 0.663, 1.041],
       [0.529, 0.809, 0.693, 0.574, 0.111, 1.06 ],
       [0.284, 0.715, 0.659, 0.664, 0.513, 0.984],
       [0.761, 0.081, 0.491, 0.758, 0.739, 1.047],
       [0.707, 0.457, 0.079, 0.655, 0.611, 0.953],
       [1.064, 1.044, 0.959, 1.103, 1.057, 0.   ]])

In [3]:
len(events)

3211

In [15]:
events_by_dates = group_by_dates(events)

In [37]:
for date, date_events in events_by_dates.items():
    events_by_place = group_by_place(date_events)
    for place, place_events in events_by_place.items():
        if len(place_events) > 1:
            texts = list(set([preprocess(e['description']) for e in place_events]))
            if len(texts) == 1:
                continue
            for t in texts:
                print(t)
                print()
            #vectors = get_vectors(texts)
            print_heat_matrix_text(texts)
            print("\n\n")

9 марта 2020 года 19.00 в концертный зал «Верхний город», концертный духовой оркестр «Немига» приглашает всех любителей популярной, эстрадной и джазовой музыки. Программа «Музыка любви» Любовь к своей родной земле, к матери, к женщине, к своим детям, своему Отечеству – вот чем живет человек на этой Земле. Этому прекрасному чувству и посвящается этот концерт. Заслуженный артист республики Республики Беларусь, настоящий полковник, почетный гражданин французских городов Суше и Бетюн, главный дирижер концертного оркестра «Светоч», доцент БГУКИ, дирижер концертного оркестра «Немига», ученик народного артиста СССР Николая Некрасова – подарит Вам новую программу в исполнении концертного оркестра «Немига» и солистки Заслуженного коллектива Республики Беларусь ГУ «образцово-показательный оркестр Вооруженных сил РБ» Полины Донской и солиста белорусской государственной филармонии Игоря Задорожного. В программе концерта Вас ждет много приятных и неожиданных сюрпризов. Превращение классических тем 

[[0.         0.91487532]
 [0.91487532 0.        ]]



Дмитрий Быков выступит в Минске с лекцией «На самом деле мне нравилась только ты. Главные стихи». 22 марта в Конгресс-Холл Президент Отеля состоятся две лекции блистательного литературоведа, писателя, журналиста, одного из авторов проекта «Гражданин поэт», а, главное, обожаемого детьми и их родителями учителя Дмитрия Быкова. 22 марта, в 18.00 состоится лекция автора «На самом деле мне нравилась только ты. Главные стихи». В рамках творческой встречи в Минске Быков поведает гостям о стихах, которые он считает для себя главными. Он прочитает поэзию и прозу, расскажет о малоизвестных моментах своей жизни и, конечно же, ответит на вопросы зрителей. Для большинства гостей творческий вечер станет настоящим откровением. Ведь слушая глубокие размышления или ироничные высказывания ДБ, мало кто задумывается, что этот автор — прежде всего лирический поэт. Гости вечера узнают нового Быкова, нежного и уязвимого. Увидят, что в стихах он не наступа

Денис Мацуев впервые даст большой сольный концерт на сцене Большого театра Беларуси. 13 апреля Минск ждёт яркое музыкальное событие. Впервые на сцене Большого театра Беларуси выступит пианист-виртуоз Денис Мацуев. Музыкант является желанным гостем центральных концертных залов мира, непременным участником крупнейших музыкальных фестивалей, постоянным партнером ведущих симфонических оркестров России, Европы, Северной Америки и стран Азии. Его выступления имеют феноменальный успех. Музыкант продолжает традиции легендарной русской фортепианной школы, а его концертные программы отличаются неизменным качеством. Уникальный музыкант современности представит минской публике программу, в основу которой легли известные произведения Петра Ильича Чайковского: «Времена года», «Размышление», большая соната Соль мажор, «Думка». Если вы хотите услышать выступление, которое британская газета «The Independent» назвала «русской мощью» и «героической силой», приходите 13 апреля в Большой театр Беларуси. Ва

[[0.         0.47511929]
 [0.4730743  0.        ]]



Новый Сезон! Больше музыки! Та самая музыка, та самая атмосфера! Лучшие Хиты 80-90х, и популярные танцевальные хиты -00х от лучших представителей жанра. Вспомни молодость вместе с нами! DJ CRASH DJ VYSOCKIY Ведущий- Артём Гоголь. ￼Вход: парням - 15р, девушкам 10р ￼фэйсконтроль/дресс код Инфо и бронирование мест: +375 29 110 35 35

Дискотека 80/90х! Новый сезон! Больше музыки! Та самая музыка, та самая атмосфера! Лучшие хиты 80-90х и популярные танцевальные Хиты -00х от лучших представителей жанра. Вспомни молодость вместе с нами! DJ CRASH / DJ VYSOCKIY Ведущий — Артём Гоголь. Вход: Парням — 15 р. Девушкам — 10 р. Инфолиния и бронирование мест: +375 29 110-35-35 клуб «RE:PUBLIC» (ул. Притыцкого, 62) FC\DC\18+

[[0.         0.78321678]
 [0.81678322 0.        ]]



При поддержке Jagermeister Belarus. В пятницу, 13 марта, в стенах клуба Hide пройдёт первая в этом году фирменная вечеринка Happy Hours от команды Znichka. На этот раз лицом

[[0.         0.06309611]
 [0.05355833 0.        ]]



«Нас поймут через 100 лет. Лазарь Хидекель»: 15 февраля в Минске откроется уникальная выставка к 100-летию художественного объединения УНОВИС В феврале 2020 года исполняется 100 лет с момента основания всемирно известного художественного объединения УНОВИС, созданного в Витебске по инициативе Казимира Малевича. В преддверии этой знаменательной даты в Беларуси стартовал масштабный проект, который реализуется Центром белорусско-еврейского культурного наследия совместно с Музеем истории Витебского народного художественного училища и компанией А1. Главным событием станет первая в истории Беларуси выставка Лазаря Хидекеля, которая откроется в Национальном художественном музее Республики Беларусь 15 февраля. «Нас поймут через 100 лет. Лазарь Хидекель» – первая в Беларуси персональная выставка первого в мире супрематического архитектора, художника, новатора и яркого представителя авангарда. Благодаря поддержке А1 специально для выставки в 

[[0.         0.17690105 0.32544181 0.96079077 0.17551669 0.28053492]
 [0.17499205 0.         0.1625     0.16440049 0.99103407 0.13236119]
 [0.2900967  0.170625   0.         0.27296248 0.16677077 0.87317358]
 [0.16540362 0.14400494 0.13648124 0.         0.14638666 0.11220359]
 [0.1736089  0.99103407 0.16114928 0.16676961 0.         0.13452915]
 [0.25973254 0.13965227 0.87317358 0.2573742  0.13957399 0.        ]]



Музыку Ghostemane рискованно включать, гуляя ночью по городу – на каждом углу мерещатся монстры и призраки. Лучше всего она заходит, если находиться в эпицентре слэма на танцполе. 15 апреля 2020 года Ghostemane приедет в Беларусь, чтобы выступить на площадке Re:Public. Как только ни пытались критики определить стиль музыки Ghostemane: блэк-метал-рэп, клауд-рэп, экстремальный или андеграундный рэп. Но эта хлесткая, громкая и пробивная гремучая смесь не нуждается в определениях, ведь она пробирает до мозга костей с первого звука. Основа каждого сингла американского музыканта Gh

[[0.         0.15656963]
 [0.15420928 0.        ]]



29 февраля нашему любимому ресторану «Ангелы» исполняется 5 лет! Будем рады провести наш день рождения вместе с вами. Вас ждет потрясающая праздничная программа: Ведущий - Андрей Ермолинский Зажигательное выступление ваших любимых артистов - Meriem Band No Comment Band Korol Project Розыгрыши призов и подарков от ресторана! Праздничный торт и море положительных эмоций! Начало в 22.00 Успейте забронировать столик уже сегодня! +375 44 575-44-44.

29 февраля нашему любимому ресторану «Ангелы» исполняется 5 лет! Будем рады провести его вместе с вами. Вас ждет потрясающая праздничная программа: Ведущий - Андрей Ермолинский Зажигательное выступление ваших любимых артистов - Meriem Band No Comment Band Korol Project Розыгрыши призов и подарков от ресторана! Праздничный торт и море положительных эмоций! Успейте забронировать столик уже сегодня! +375 44 575 44 44

[[0.         0.94444444]
 [0.94444444 0.        ]]



29 февраля в 22:00 перва

In [33]:
scipy.spatial.distance.cosine([-11, 32, 322], [1, 1, 1])

0.38836437773614707